In [5]:
import pandas as pd
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import datapath

import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
from itertools import chain

import warnings
warnings.simplefilter("ignore")

In [8]:
lda = gensim.models.ldamodel.LdaModel
stop = set(stopwords.words('english')).union(stopwords.words('spanish'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

temp_file = datapath("dictio")
dictionary = corpora.Dictionary.load(temp_file, mmap='r')

temp_file = datapath("text_clustering_model")
ldamodel = lda.load(temp_file, mmap='r')


def clean(text):
    stop_free = " ".join([word for word in text.lower().split() if word not in stop])
    punct_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join([lemma.lemmatize(word) for word in punct_free.split()])
    return normalized.split()

def cluster_text(text, umbral_param = 0.6):
    clean_text = [clean(text)]
    doc_term_matrix_test = [dictionary.doc2bow(doc) for doc in clean_text]
    distribution = ldamodel[doc_term_matrix_test[0]]
    max_ = get_max(distribution, umbral_param)
    return max_

def get_max(list_distribution, umbral_param):
    umbral = umbral_param
    temp_label = -1
    temp_max = 0
    for cluster in list_distribution:
        if cluster[1] > temp_max and cluster[1] > umbral:
            temp_max = cluster[1]
            temp_label = cluster[0]
    return temp_label

In [4]:
text = "ciencia investigacion matemáticas física quimica"
text2 = "Mira a miles de personas reales desnudas y teniendo sexo en directo en webcam las 24 horas. 100% Gratis para mirar, chatear y transmitir. No se necesita ..."
cluster_text(text2)

2